# Single molecule - Manual Pipeline

____
This code is intended to take a video (.tiff) from fluorescence microscopy, segment the cell,detect single-RNA spots, track the single-RNA spots in time, and quantify spot intensity.
____
The code is divided in the following sections:

* Alignment images taken from the two cameras.
* Load microscope video.
* Mask selection. Manual selection of a Region Of Interest (ROI).
* Particle tracking using trackpy. 
____

## Loading libraries and images
___

In [ ]:
from skimage.io import imread
import numpy as np 
import pathlib
import sys
import matplotlib.pyplot as plt 
import cv2
import trackpy as tp
import ipywidgets as widgets                       # Library to plot widgets
from ipywidgets import interact, interactive, HBox, Layout, VBox #  importing modules and functions.


In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
video_dir = current_dir.parents[1].joinpath('DataBases','test_videos','ran_short','MAX_Cell01.tif')
beads_dir = current_dir.parents[1].joinpath('DataBases','test_videos','ran_short_beads','Beads01.tif')

In [ ]:
# Importing rSNAPed
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp

In [ ]:
# Loading images
image_with_beads = imread(beads_dir)
video = imread(video_dir)

# Video registration
___

In [ ]:
# Visualizing images with beads
print('The dimensions in the image with beads are: ', image_with_beads.shape, '\n')
# Visualizing the beads
fig, ax = plt.subplots(2,2, figsize=(10, 10))
ax[0,0].imshow(image_with_beads[0,:,:],cmap=plt.cm.Reds_r)
ax[0,0].set_xticks([]); ax[0,0].set_yticks([])
ax[0,1].hist(image_with_beads[0,:,:].flatten(), bins=80,color='red', ec="cyan")
ax[0,1].set_xlabel('Intensity'); ax[0,1].set_ylabel('Count')
ax[1,0].imshow(image_with_beads[1,:,:],cmap=plt.cm.Greens_r)
ax[1,0].set_xticks([]); ax[1,0].set_yticks([])
ax[1,1].hist(image_with_beads[1,:,:].flatten(), bins=80,color='green', ec="cyan")
ax[1,1].set_xlabel('Intensity'); ax[1,1].set_ylabel('Count')
plt.show()

In [ ]:
# Calculating the homography matrix
homography_matrix = rsp.BeadsAlignment(first_image_beads= image_with_beads[0,:,:] ,second_image_beads = image_with_beads[1,:,:], spot_size = 5, min_intensity = 300).make_beads_alignment()

In [ ]:
# Loading images with the cells
print('The dimensions in the video are: ', video.shape, '\n')
# Plotting the beads
selected_time_point = 0
fig, ax = plt.subplots(3,2, figsize=(10, 10))
ax[0,0].imshow(video[selected_time_point,:,:,0],cmap=plt.cm.Reds_r)
ax[0,0].set_title('Channel 0 = RNA');ax[0,0].set_xticks([]); ax[0,0].set_yticks([])
ax[0,1].hist(video[selected_time_point,:,:,0].flatten(), bins=80,color='red', ec="cyan")
ax[0,1].set_xlabel('Intensity'); ax[0,1].set_ylabel('Count')
ax[1,0].imshow(video[selected_time_point,:,:,1],cmap=plt.cm.Greens_r)
ax[1,0].set_title('Channel 1 = Protein 1'); ax[1,0].set_xticks([]); ax[1,0].set_yticks([])
ax[1,1].hist(video[selected_time_point,:,:,1].flatten(), bins=80,color='green', ec="cyan")
ax[1,1].set_xlabel('Intensity'); ax[1,1].set_ylabel('Count')
ax[2,0].imshow(video[selected_time_point,:,:,2],cmap=plt.cm.Blues_r)
ax[2,0].set_title('Channel 2 = Protein 2'); ax[2,0].set_xticks([]); ax[2,0].set_yticks([])
ax[2,1].hist(video[selected_time_point,:,:,2].flatten(), bins=80,color='blue', ec="cyan")
ax[2,1].set_xlabel('Intensity'); ax[2,1].set_ylabel('Count')
plt.show()

In [ ]:
# Applying the alignment transformation to the whole video. Matrix multiplication to align the images from the two cameras.
transformed_video = rsp.CamerasAlignment(video, homography_matrix, target_channels= [1,2]).make_video_alignment()

In [ ]:
# Plotting the beads
selected_time_point = 0
fig, ax = plt.subplots(3,2, figsize=(10, 10))
ax[0,0].imshow(transformed_video[selected_time_point,:,:,0],cmap=plt.cm.Reds_r)
ax[0,0].set_title('Channel 0 = RNA');ax[0,0].set_xticks([]); ax[0,0].set_yticks([])
ax[0,1].hist(transformed_video[selected_time_point,:,:,0].flatten(), bins=80,color='red', ec="cyan")
ax[0,1].set_xlabel('Intensity'); ax[0,1].set_ylabel('Count')
ax[1,0].imshow(transformed_video[selected_time_point,:,:,1],cmap=plt.cm.Greens_r)
ax[1,0].set_title('Channel 1 = Protein 1'); ax[1,0].set_xticks([]); ax[1,0].set_yticks([])
ax[1,1].hist(transformed_video[selected_time_point,:,:,1].flatten(), bins=80,color='green', ec="cyan")
ax[1,1].set_xlabel('Intensity'); ax[1,1].set_ylabel('Count')
ax[2,0].imshow(transformed_video[selected_time_point,:,:,2],cmap=plt.cm.Blues_r)
ax[2,0].set_title('Channel 2 = Protein 2'); ax[2,0].set_xticks([]); ax[2,0].set_yticks([])
ax[2,1].hist(transformed_video[selected_time_point,:,:,2].flatten(), bins=80,color='blue', ec="cyan")
ax[2,1].set_xlabel('Intensity'); ax[2,1].set_ylabel('Count')
plt.show()

# Selecting a region of interest (a.k.a. Mask) in the image
____

In [ ]:
###########
# Magic function to interact with the figure.
%matplotlib widget 
###########
# Selecting the first time point and first channel
on1= rsp.ManualMask(video=transformed_video,time_point_selected=0,selected_channel=1)  # NOTICE THAT FOR THE MASK SELECTION WE USE THE CHANNEL 1 (GREEN CHANNEL)

In [ ]:
###########
# Magic function to have the figure as an output in the notebook
%matplotlib inline 
###########
arr = np.array([on1.selected_points],'int')
mask = cv2.fillPoly(np.zeros(video[0,:,:,1].shape,np.uint8),arr,[1,1,1])
plt.figure( figsize=(5, 5))
plt.imshow(mask)
plt.show()

# Particle tracking using Trackpy

____


The particle tracking is performed using [trackpy](http://http://soft-matter.github.io/trackpy/v0.5.0/)
The library documentation can be accessed in the following [link](https://buildmedia.readthedocs.org/media/pdf/trackpy/v0.2.3/trackpy.pdf).

### Calculating the intensity of the detected particles.

In [ ]:
# This section generates an histogram with the intensity of the detected particles in the video.
particle_size = 7 # according to the documentation must be an odd number 3,5,7,9 etc.
minimal_intensity_for_selection = 0 # minimal intensity to detect a particle.
selected_channel_for_tracking = 0

# "f" is a pandas data frame that contains the information about the detected spots
f = tp.locate(transformed_video[0,:,:,selected_channel_for_tracking], particle_size, minmass=minimal_intensity_for_selection) 
print('All particles of size: ',  str(particle_size))
plt.figure( figsize=(5,5))
tp.annotate(f,transformed_video[0,:,:,selected_channel_for_tracking],color='yellow',plot_style={'markersize': 1})
plt.show()

print('Intensity distribution for all particles of size: ',  str(particle_size))
fig, ax = plt.subplots(figsize=(5, 5))
ax.hist(f['mass'], bins=100, color = "orangered", ec="cyan")
ax.set(xlabel="Particle's 'mass' ", ylabel='count')
plt.show()

### Select an intensity threshold value and particle size

In the following widget, the user is asked to manually input the particle size and the Intensity threshold, these numbers will be used for during the rest of the code to calculate intensities. Notice that tracking is only performed on channel 0.

In [ ]:
# To start visualization move the time slider.
plt.rcParams["figure.figsize"] = (5,5) # if movie is too big, change size to (7,7)
def figure_viewer_tr(time,mass_text, particle_size):
    ch = 0  
    f = tp.locate(transformed_video[time,:,:,ch],particle_size, minmass=mass_text, maxsize=7, percentile=60) # "f" is a pandas data frame that contains the information about the detected spots
    tp.annotate(f,transformed_video[time,:,:,ch]) # tp.annotate is a trackpy function that displays the image with the detected spots  
values_size=[3,5,7,9] # Notice value must be an odd number.
interactive_plot_tr = interactive(figure_viewer_tr, mass_text = widgets.IntSlider(value=400,min=100,max=2000,description='min Intensity'), particle_size = widgets.IntSlider(min=3,max=11,step=2,value=7,description='Particle Size'), time = widgets.IntSlider(min=0,max=video.shape[0]-1,step=1,value=0,description='Time'))
controls = HBox(interactive_plot_tr.children[:-1], layout = Layout(flex_flow='row wrap'))
output = interactive_plot_tr.children[-1]
display(VBox([controls, output])) 

### Saving the threshold parameters that will be used to define the spot selection criteria

In [ ]:
# This section saves the parameters adjusted in the previous widget in two variables that will be use for the rest of the code.
selected_intensity = interactive_plot_tr.kwargs_widgets[1].value
selected_size = interactive_plot_tr.kwargs_widgets[2].value

### Detecting the spots in all frames

In [ ]:
# "f" is a pandas data frame that contains the information about the detected spots.
# tp.batch is a trackpy function that detects spots for multiple frames in a video.
f = tp.batch(video[:,:,:,0],selected_size, minmass=selected_intensity,percentile=70)
f

### Removing particles outside the mask

In [ ]:
def spots_in_mask(f,mask):
    # extracting the contours in the image
    coords = np.array([f.y, f.x]).T # These are the points detected by trackpy
    coords_int = np.round(coords).astype(int)  # or np.floor, depends
    values_at_coords = mask[tuple(coords_int.T)] # If 1 the value is in the mask
    f['In Mask']=values_at_coords # Check if pts are on/in polygon mask  
    return f 

In [ ]:
f1 = spots_in_mask(f,mask) 
f1

In [ ]:
dataframe_particles_in_mask = f[f['In Mask']==True]
dataframe_particles_in_mask

### Linking all detected spots across all frames

In [ ]:
# tp.link_df is a trackpy function that links spots detected in multiple frames, this generates the spots trajectories in time.
df_linked_particles = tp.link_df(dataframe_particles_in_mask,5, memory=2) # tp.link_df(data_frame, max_distance_particle_moves, max_time_particle_vanishes).

### Eliminating spurious trajectories. 

In [ ]:
#trackpy.filtering.filter_stubs(tracks, threshold=100)
df_linked_particles_filtered = tp.filter_stubs(df_linked_particles, 2)  # selecting trajectories that appear in at least 10 frames.
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', df_linked_particles['particle'].nunique())
print('After:', df_linked_particles_filtered['particle'].nunique())

In [ ]:
# Optional section that saves the particles trajectories and intensities as a CSV file
df_linked_particles_filtered.to_csv('tracking_particles.csv', index = False)

## Working with trajectories
____

In [ ]:
# Extracting the intensity values for particles with intensity higher than a threshold and in frame 0
df_linked_particles_filtered.loc[(df_linked_particles_filtered['mass']>=1000) & (df_linked_particles_filtered['frame']==0)  ]

### Plotting trajectories in x-y.

In [ ]:
# Plotting 
plt.figure(figsize=(5,5))
for i in range(0,df_linked_particles_filtered['particle'].nunique() ):
    x_val = df_linked_particles_filtered.loc[df_linked_particles_filtered['particle']==i ].x.values[:]
    y_val = df_linked_particles_filtered.loc[df_linked_particles_filtered['particle']==i ].y.values[:]
    plt.plot(x_val,y_val, '-',linewidth = 3 )
plt.xlabel('x-axis')
plt.ylabel('y-axis')
plt.title('Particle movement 2D')
plt.show()

### Plotting trajectories versus intensity using trackpy

In [ ]:
# Defining the number of detected spots.
n_particles = df_linked_particles_filtered['particle'].nunique()

# Plotting
plt.figure(figsize=(10,5))
for id in range(0,n_particles):
    plt.plot(df_linked_particles_filtered.loc[df_linked_particles_filtered['particle']==df_linked_particles_filtered['particle'].unique()[id]].frame, df_linked_particles_filtered.loc[df_linked_particles_filtered['particle']==df_linked_particles_filtered['particle'].unique()[id]].mass )
plt.xlabel('Time (sec)')
plt.ylabel('Intensity a.u.')
plt.title('Intensity vs Time')
plt.show()